In [ ]:
from langchain.tools import tool

@tool
def calculate_sum(a: int, b: int) -> int:
    """计算两个整数的和"""
    return a + b

@tool
def get_weather(city: str) -> str:
    """获取指定城市的天气信息"""
    return f"{city}今天晴天，温度25°C"

In [19]:
tool_instance = calculate_sum

print(f"工具名称: {tool_instance.name}")
print(f"工具描述: {tool_instance.description}")
print(f"参数模式: {tool_instance.args}")

工具名称: calculate_sum
工具描述: 计算两个整数的和
参数模式: {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


### 自定义工具名称和描述

In [ ]:
@tool("custom_calculator", description="自定义计算器工具")
def multiply(x: float, y: float) -> float:
    """计算两个浮点数的乘积"""
    return x * y

### 复杂参数

In [ ]:
from typing import List
from pydantic import Field

@tool
def process_numbers(numbers: List[int], operation: str = "sum") -> float:
    """处理数字列表，支持求和、平均值等操作
    
    Args:
        numbers: 数字列表
        operation: 操作类型 (sum, avg, max, min)
    """
    if operation == "sum":
        return sum(numbers)
    elif operation == "avg":
        return sum(numbers) / len(numbers)
    elif operation == "max":
        return max(numbers)
    elif operation == "min":
        return min(numbers)
    else:
        return 0

### 与Agent集成

In [ ]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain.tools import tool

@tool
def calculate_sum(a: int, b: int) -> int:
    """计算两个整数的和"""
    return a + b

@tool
def get_weather(city: str) -> str:
    """获取指定城市的天气信息"""
    return f"{city}今天晴天，温度25°C"

# 创建工具列表
tools = [calculate_sum, get_weather]

# 初始化Agent
agent = create_agent(
    model="gpt-4o",
    tools=tools
)

# 使用Agent
result = agent.invoke({"messages": [HumanMessage(content="计算15和25的和，然后查询北京的天气")]})

print(result["messages"][-1].content)

# 15和25的和是40。  
# 北京的天气今天是晴天，温度为25°C。

## 自定义参数模式 (args_schema)

### Pydantic

In [27]:
from pydantic import BaseModel, Field
from typing import Literal

class WeatherInput(BaseModel):
    """天气查询的输入。"""
    location: str = Field(description="城市名称或坐标")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="温度单位偏好"
    )
    include_forecast: bool = Field(
        default=False,
        description="包含5天预报"
    )

@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """获取当前天气和可选预报。"""
    temp = 22 if units == "celsius" else 72
    result = f"{location}的当前天气：{temp}度{units[0].upper()}"
    if include_forecast:
        result += "\n未来5天：晴天"
    return result

# 初始化Agent
agent = create_agent(
    model="gpt-4o",
    tools=[get_weather]
)

# 使用Agent - 提供明确的参数
result = agent.invoke({"messages": [HumanMessage(content="查询北京将来几天的天气")]})
print(result["messages"][-1].content)

北京目前的天气是22°C，未来5天的预报显示都是晴天。


### JSON Schema

In [26]:
weather_schema = {
    "type": "object",
    "properties": {
        "location": {"type": "string"},
        "units": {"type": "string"},
        "include_forecast": {"type": "boolean"}
    },
    "required": ["location"]
}

@tool(args_schema=weather_schema)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """获取指定城市的天气信息"""
    print("location：", location)
    print("units：", units)
    print("include_forecast：", include_forecast)

    temp = 22 if units == "celsius" else 72
    result = f"{location}的当前天气：{temp}度{units[0].upper()}"
    if include_forecast:
        result += "\n未来5天：晴天"
    return result


# 初始化Agent
agent = create_agent(
    model="gpt-4o",
    tools=[get_weather]
)

# 使用Agent - 提供明确的参数
result = agent.invoke({"messages": [HumanMessage(content="查询北京将来几天的天气")]})
print(result["messages"][-1].content)

location： 北京
units： celsius
include_forecast： True
北京目前的气温是22°C，将来的天气情况预计为晴天，持续未来5天。


## Accessing Context

### Context

#### 读取

In [46]:
# 简单的状态修改案例
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.messages import HumanMessage

# 全局状态存储
simple_state = {
    "counter": 666
}

# 读取状态的工具
@tool
def get_counter() -> str:
    """获取当前计数器值"""
    return f"当前计数器: {simple_state['counter']}"

result = agent.invoke(
    {"messages": [
        HumanMessage("读取计数器的值")
    ]}
)

print(result["messages"][-1].content)

print(simple_state)

当前计数器的值是 666。
{'counter': 666}


#### 修改

In [45]:
# 简单的状态修改案例
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.messages import HumanMessage

# 全局状态存储
simple_state = {
    "counter": 666
}

# 修改状态的工具
@tool
def increment_counter(num: int) -> str:
    """增加计数器"""
    simple_state['counter'] += num
    return f"计数器已增加到: {simple_state['counter']}"

# 创建Agent并测试
agent = create_agent(
        model="gpt-4o", 
        tools=[get_counter, increment_counter, set_user_name]
    )

result = agent.invoke(
    {"messages": [
        HumanMessage("计数器增加222")
    ]}
)

print(result["messages"][-1].content)

print(simple_state)

计数器已增加222，现在的计数器值是888。
{'counter': 888}


#### 多用户

In [50]:
from dataclasses import dataclass
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime

# 简化的用户数据库
USER_DATABASE = {
    "user123": {"name": "Alice", "balance": 1000},
    "user456": {"name": "Bob", "balance": 500}
}

@dataclass
class UserContext:
    user_id: str

# 查询余额工具
@tool
def get_balance(runtime: ToolRuntime[UserContext]) -> str:
    """查询当前用户的账户余额"""
    user_id = runtime.context.user_id
    
    if user_id in USER_DATABASE:
        user = USER_DATABASE[user_id]
        return f"{user['name']}的余额：${user['balance']}"
    return "用户不存在"

# 存款工具
@tool
def deposit(amount: int, runtime: ToolRuntime[UserContext]) -> str:
    """向当前用户账户存款"""
    if amount <= 0:
        return "存款金额必须大于0"
    
    user_id = runtime.context.user_id
    
    if user_id in USER_DATABASE:
        USER_DATABASE[user_id]["balance"] += amount
        return f"成功存款${amount}，当前余额：${USER_DATABASE[user_id]['balance']}"
    return "用户不存在"

# 创建Agent
tools = [get_balance, deposit]

agent = create_agent(
    model="gpt-4o",
    tools=tools,
    context_schema=UserContext,
    system_prompt="你是一个简单的银行助手。"
)

# 简化测试
print("=== 简化金融助手演示 ===")

# 1. 查询初始余额
print("\n1. 查询余额:")
result = agent.invoke(
    {"messages": [{"role": "user", "content": "查询我的余额"}]},
    context=UserContext(user_id="user123")
)
print(result["messages"][-1].content)

# 2. 存款
print("\n2. 存款200:")
result = agent.invoke(
    {"messages": [{"role": "user", "content": "我要存款200"}]},
    context=UserContext(user_id="user123")
)
print(result["messages"][-1].content)

# 3. 再次查询余额
print("\n3. 再次查询余额:")
result = agent.invoke(
    {"messages": [{"role": "user", "content": "现在我的余额是多少"}]},
    context=UserContext(user_id="user123")
)
print(result["messages"][-1].content)

print(f"\n最终数据库状态: {USER_DATABASE}")

=== 简化金融助手演示 ===

1. 查询余额:
您的账户余额是 $1000。

2. 存款200:
您已成功存款 $200，现在您的账户余额为 $1200。

3. 再次查询余额:
您的当前账户余额是 $1200。

最终数据库状态: {'user123': {'name': 'Alice', 'balance': 1200}, 'user456': {'name': 'Bob', 'balance': 500}}


### State（缺失）

=== ToolRuntime 状态访问演示 ===

1. 总结当前对话:
到目前为止，我们的对话包含了一条用户消息和一条AI响应，没有涉及工具使用。用户请求总结对话内容，而我尝试获取总结信息。


### Memory (Store) （缺失）

In [ ]:
from typing import Any
from langgraph.store.memory import InMemoryStore
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime

# 访问记忆
@tool
def get_user_info(user_id: str, runtime: ToolRuntime) -> str:
    """查找用户信息。"""
    store = runtime.store
    user_info = store.get(("users",), user_id)
    return str(user_info.value) if user_info else "未知用户"

# 更新记忆
@tool
def save_user_info(user_id: str, user_info: dict[str, Any], runtime: ToolRuntime) -> str:
    """保存用户信息。"""
    store = runtime.store
    store.put(("users",), user_id, user_info)
    return "成功保存用户信息。"

store = InMemoryStore()
agent = create_agent(
    model,
    tools=[get_user_info, save_user_info],
    store=store
)

# 第一会话：保存用户信息
agent.invoke({
    "messages": [{"role": "user", "content": "保存以下用户：用户ID：abc123，姓名：Foo，年龄：25，邮箱：foo@langchain.dev"}]
})

# 第二会话：获取用户信息
agent.invoke({
    "messages": [{"role": "user", "content": "获取ID为'abc123'的用户信息"}]
})


### Stream Writer（缺失）

In [ ]:
from langchain.tools import tool, ToolRuntime

@tool
def get_weather(city: str, runtime: ToolRuntime) -> str:
    """获取给定城市的天气。"""
    writer = runtime.stream_writer

    # 工具执行时流式更新
    writer(f"正在查找城市数据: {city}")
    writer(f"已获取城市数据: {city}")

    return f"{city} 总是晴天！"